## SQL Kaggle Tutorial

## SQLite Commands

SELECT -> Used to query data from one or more table

COUNT -> Aggregate function that returns the number of items in a group. If uses the ALL argument, all non-null values included duplicates will be counted. If DISTINCT specified, only the unique and non-null values will bem counted

DISTINCT -> Query unique rows in a table (Remove duplicate rows in the result set).
Consider NULL values as duplicates. If used with a column with NULL values, SQLite will keep one row of a NULL value.

GROUP BY -> Get the group rows into groups and apply aggregate function for each group

FROM -> Specifies the table from which the data will be taken

ORDER BY -> Allows to sort the result set based on one or more columns in ascending or descending order. Comes after the FROM. (ASC = Ascending. DESC = Descending). Uses ascending order by default.

UNION -> Combine rows from two or more queries into a single result set. UNION ALL operator does not remove duplicate rows. UNION operator removes/eliminate duplicate rows

JOIN -> Query data from two or more tables

#### Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import os

## Connecting to SQL Database

#### Create conection

In [3]:
connection = sql.connect("basketball.sqlite")
print("SQL database connected")

SQL database connected


#### Show all the tables title

In [4]:
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

                     name
0                  Player
1                    Team
2         Team_Attributes
3            Team_History
4       Player_Attributes
5          Game_Officials
6   Game_Inactive_Players
7             Team_Salary
8           Player_Salary
9                   Draft
10          Draft_Combine
11          Player_Photos
12            Player_Bios
13                   Game
14                   News
15           News_Missing


## Drafting Data Analysis

#### Drafting count change over time from 1949 to 2020

In [5]:
df = pd.read_sql_query("SELECT * FROM Draft", connection)
df

,yearDraft,numberPickOverall,numberRound,numberRoundPick,namePlayer,slugTeam,nameOrganizationFrom,typeOrganizationFrom,idPlayer,idTeam,nameTeam,cityTeam,teamName,PLAYER_PROFILE_FLAG,slugOrganizationTypeFrom,locationOrganizationFrom
0,2020.0,1.0,1.0,1.0,Anthony Edwards,MIN,Georgia,College/University,1630162.0,1.610613e+09,Minnesota Timberwolves,Minnesota,Timberwolves,1.0,COL,None
1,2020.0,2.0,1.0,2.0,James Wiseman,GSW,Memphis,College/University,1630164.0,1.610613e+09,Golden State Warriors,Golden State,Warriors,1.0,COL,None
2,2020.0,3.0,1.0,3.0,LaMelo Ball,CHA,Illawarra,Other Team/Club,1630163.0,1.610613e+09,Charlotte Hornets,Charlotte,Hornets,1.0,PRO,Australia
3,2020.0,4.0,1.0,4.0,Patrick Williams,CHI,Florida State,College/University,1630172.0,1.610613e+09,Chicago Bulls,Chicago,Bulls,1.0,COL,None
4,2020.0,5.0,1.0,5.0,Isaac Okoro,CLE,Auburn,College/University,1630171.0,1.610613e+09,Cleveland Cavaliers,Cleveland,Cavaliers,1.0,COL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885,1949.0,6.0,1.0,6.0,Ron Livingstone,BAL,Wyoming,College/University,77392.0,1.610610e+09,Baltimore Bullets,Baltimore,Bullets,1.0,COL,None
7886,1949.0,7.0,1.0,7.0,Dick McGuire,NYK,St. John's (NY),College/University,77537.0,1.610613e+09,New York Knicks,New York,Knicks,1.0,COL,None
7887,1949.0,8.0,1.0,8.0,Wallace Jones,WAS,Kentucky,College/University,77199.0,1.610610e+09,Washington Capitols,Washington,Capitols,1.0,COL,None
7888,1949.0,9.0,1.0,9.0,Jack Kerris,CHS,Loyola-Chicago,College/University,77249.0,1.610610e+09,Chicago Stags,Chicago,Stags,1.0,COL,None


## Get data with SQL

In [6]:
query = """
        SELECT
            yearDraft as year_drafted,
            COUNT(DISTINCT idPlayer) AS total_drafted
        FROM Draft
        GROUP BY yearDraft;
"""

year_draft_total = pd.read_sql(query, connection).astype({"year_drafted": int})
year_draft_total

,year_drafted,total_drafted
0,1949,75
1,1950,121
2,1951,87
3,1952,106
4,1953,122
...,...,...
67,2016,60
68,2017,60
69,2018,60
70,2019,60


In [7]:
px.line(year_draft_total, x="year_drafted", y="total_drafted",
        title="NBA Drafting Trend from 1949 to 2020")

#### What are the top 10 NBA teams that drafted most number of players from university

In [8]:
query = """
        SELECT
            nameTeam as nba_team,
            COUNT(DISTINCT idPlayer) as total_drafted
        FROM Draft
        Group BY idTeam
        ORDER BY total_drafted DESC
        LIMIT 10;
"""

nba_team_draft_total = pd.read_sql(query, connection)
nba_team_draft_total

,nba_team,total_drafted
0,Atlanta Hawks,514
1,Sacramento Kings,511
2,Philadelphia 76ers,487
3,New York Knicks,474
4,Golden State Warriors,451
5,Boston Celtics,448
6,Detroit Pistons,431
7,Los Angeles Lakers,426
8,Washington Wizards,411
9,Chicago Bulls,341


#### When did the teams first started to draft players fro the universities?

In [9]:
query = """
        SELECT
            MIN(yearDraft) as first_draft_year,
            nameTeam as nba_team,
            COUNT(DISTINCT yearDraft) AS years_drafted,
            COUNT(DISTINCT idPlayer) AS total_drafted_to_date
        FROM Draft
        GROUP BY idTeam
        ORDER BY first_draft_year;
"""

nba_team_first_draft_year = pd.read_sql(query, connection)
nba_team_first_draft_year.head()

,first_draft_year,nba_team,years_drafted,total_drafted_to_date
0,1949.0,Baltimore Bullets,6,68
1,1949.0,Chicago Stags,2,13
2,1949.0,Indianapolis Olympians,4,36
3,1949.0,Providence Steamrollers,1,9
4,1949.0,St. Louis Bombers,1,10


#### For each year, how many team first started drafting new players?

In [10]:
nba_team_first_draft_year["first_draft_year"].value_counts()

1949.0    12
1977.0     4
1970.0     3
1950.0     2
1967.0     2
1968.0     2
1988.0     2
1989.0     2
1995.0     2
1961.0     1
1966.0     1
1974.0     1
1980.0     1
2003.0     1
Name: first_draft_year, dtype: int64

#### Top 5 teams with the most years of drafts

In [11]:
nba_team_first_draft_year.sort_values("years_drafted", ascending=False).head()

,first_draft_year,nba_team,years_drafted,total_drafted_to_date
6,1949.0,Boston Celtics,72,448
11,1949.0,Ft. Wayne Zollner Pistons,72,431
13,1950.0,Syracuse Nationals,71,487
8,1949.0,Minneapolis Lakers,71,426
10,1949.0,Rochester Royals,71,511


#### Do most of the players coming from high school, university or from other professional basketball team?

In [12]:
query = """
        SELECT
            typeOrganizationFrom AS organization,
            COUNT (DISTINCT idPLayer) AS player_count
        FROM Draft
        GROUP BY typeOrganizationFrom;
"""

draft_player_org = pd.read_sql(query, connection)
draft_player_org

,organization,player_count
0,None,15
1,College/University,7389
2,High School,48
3,Other Team/Club,304


In [13]:
# Didn't including the None answers

px.pie(draft_player_org.iloc[1:,], values="player_count", names="organization",
       hole=0.3, title="Drafted player organization breakdown")

#### Trend Line

In [14]:
query = """
        SELECT
            yearDraft AS year_drafted,
            typeOrganizationFrom AS organization,
            COUNT (DISTINCT idPLayer) AS drafted_count
        FROM Draft
        GROUP BY yearDraft, typeOrganizationFrom;
"""

draft_player_org_trend = pd.read_sql(query, connection).astype({"year_drafted": int})
draft_player_org_trend

,year_drafted,organization,drafted_count
0,1949,College/University,75
1,1950,College/University,121
2,1951,College/University,87
3,1952,College/University,106
4,1953,None,6
...,...,...,...
136,2019,High School,1
137,2019,Other Team/Club,7
138,2020,College/University,49
139,2020,High School,1


In [15]:
px.line(draft_player_org_trend.dropna(), x='year_drafted', y='drafted_count', color='organization',
        title='NBA drafting trend by player source organization type from 1949 to 2020')

## Game Play Data Analysis

#### From 1949 to 2020, how did the total number of participated teams count changes and how did the total game count changes along with it overall?

In [16]:
# make subplot for dual axes plot
from plotly.subplots import make_subplots

In [17]:
query = """
        SELECT
            SEASON_ID-20000 AS season,
            COUNT (DISTINCT TEAM_ID_HOME) AS total_team_count,
            COUNT (DISTINCT GAME_ID) AS total_game_count
        FROM Game
        GROUP BY SEASON_ID
"""

team_and_game_count = pd.read_sql(query, connection)
team_and_game_count

,season,total_team_count,total_game_count
0,1946,11,331
1,1947,8,194
2,1948,12,360
3,1949,17,563
4,1950,11,354
...,...,...,...
70,2016,30,1230
71,2017,30,1230
72,2018,30,1230
73,2019,30,1059


#### Create visualization with two y axis for team and game total from 1946 to 2020

In [18]:
two_y_axis_plot = make_subplots(specs=[[{"secondary_y": True}]])
team_count_trace = px.bar(team_and_game_count, x="season", y="total_team_count")
game_count_trace = px.line(team_and_game_count, x="season", y="total_game_count")
team_count_trace.update_traces(name="team totals", showlegend = True, opacity=0.6)
game_count_trace.update_traces(name="game totals", showlegend = True, line_color="red")
two_y_axis_plot.add_trace(team_count_trace.data[0], secondary_y=False)
two_y_axis_plot.add_trace(game_count_trace.data[0], secondary_y=True)
two_y_axis_plot.update_yaxes(title_text="team totals", secondary_y=False)
two_y_axis_plot.update_yaxes(title_text="game totals", secondary_y=True)
two_y_axis_plot.update_layout(title_text="1946 to 2020: NBA total team and total game play trend")

#### Verifying Game Table

In [19]:
df_Game = pd.read_sql_query("SELECT * FROM Game", connection)
df_Game

,GAME_ID,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,...,LAST_GAME_VISITOR_TEAM_CITY1,LAST_GAME_VISITOR_TEAM_POINTS,HOME_TEAM_WINS,HOME_TEAM_LOSSES,SERIES_LEADER,VIDEO_AVAILABLE_FLAG,PT_AVAILABLE,PT_XYZ_AVAILABLE,HUSTLE_STATUS,HISTORICAL_STATUS
0,0024600001,21946,1610610035,HUS,Toronto Huskies,1946-11-01,HUS vs. NYK,L,0,25.0,...,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0
1,0024600003,21946,1610610034,BOM,St. Louis Bombers,1946-11-02,BOM vs. PIT,W,0,20.0,...,None,None,5.0,1.0,St. Louis,0.0,0.0,0.0,0.0,1.0
2,0024600004,21946,1610610025,CHS,Chicago Stags,1946-11-02,CHS vs. NYK,W,0,21.0,...,None,None,3.0,3.0,Tied,0.0,0.0,0.0,0.0,1.0
3,0024600002,21946,1610610032,PRO,Providence Steamrollers,1946-11-02,PRO vs. BOS,W,0,21.0,...,None,None,5.0,1.0,Providence,0.0,0.0,0.0,0.0,1.0
4,0024600005,21946,1610610028,DEF,Detroit Falcons,1946-11-02,DEF vs. WAS,L,0,10.0,...,None,None,1.0,5.0,Washington,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62443,0022001076,22020,1610612757,POR,Portland Trail Blazers,2021-05-16,POR vs. DEN,W,240,46.0,...,DEN,106,1.0,2.0,Denver,1.0,0.0,0.0,1.0,0.0
62444,0022001071,22020,1610612750,MIN,Minnesota Timberwolves,2021-05-16,MIN vs. DAL,W,240,49.0,...,DAL,128,1.0,2.0,Dallas,1.0,0.0,0.0,0.0,0.0
62445,0022001068,22020,1610612741,CHI,Chicago Bulls,2021-05-16,CHI vs. MIL,W,240,45.0,...,MIL,108,1.0,2.0,Milwaukee,1.0,0.0,0.0,1.0,0.0
62446,0022001072,22020,1610612740,NOP,New Orleans Pelicans,2021-05-16,NOP vs. LAL,L,240,37.0,...,LAL,111,1.0,2.0,L.A. Lakers,1.0,0.0,0.0,0.0,0.0


#### From 1949 to 2020, how did home game won percentage among all the games change over time?

In [20]:
query = """
        SELECT
            SEASON_ID-20000 AS season,
            SUM (CASE WL_HOME
                    WHEN 'W' THEN 1
                    ELSE 0
                END) AS home_win_count
        FROM Game
        GROUP BY SEASON_ID
"""

home_game_stats = pd.read_sql(query, connection)
home_game_stats

,season,home_win_count
0,1946,204
1,1947,111
2,1948,228
3,1949,392
4,1950,265
...,...,...
70,2016,719
71,2017,715
72,2018,729
73,2019,584


#### Calculate game won percentage for each season

In [21]:
# Add a new column to the DataFrame

home_game_stats["home_won_percentage"] = round(100*home_game_stats["home_win_count"]/team_and_game_count["total_game_count"], 2)


In [22]:
px.line(home_game_stats, y='home_won_percentage', x='season',
        title='1946-2020: Overall home game won percentage')

#### Home game win count

In [23]:
# Each row of data representes one game for both home and away team
# Applied union to the data
# Union the HOME and AWAY Data

query = """
    SELECT
        SEASON_ID-20000 AS season,
        TEAM_ID_HOME as team_id,
        TEAM_NAME_HOME AS team_name,
        SUM(CASE WL_HOME
                WHEN 'W' THEN 1
                ELSE 0
            END) AS win_count,
        COUNT(TEAM_ID_HOME) AS team_game_count,
        "home" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_HOME

    UNION

    SELECT
        SEASON_ID-20000 AS season,
        TEAM_ID_AWAY AS team_id,
        TEAM_NAME_AWAY AS team_name,
        SUM(CASE WL_AWAY
                WHEN 'W' THEN 1
                ELSE 0
            END) AS win_count,
        COUNT(TEAM_ID_AWAY) AS team_game_count,
        "away" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_AWAY
"""

team_level_home_game_stats = pd.read_sql(query, connection)
team_level_home_game_stats

,season,team_id,team_name,win_count,team_game_count,game_location
0,1946,1610610025,Chicago Stags,17,30,away
1,1946,1610610025,Chicago Stags,22,31,home
2,1946,1610610026,Cleveland Rebels,13,30,away
3,1946,1610610026,Cleveland Rebels,17,30,home
4,1946,1610610028,Detroit Falcons,8,30,away
...,...,...,...,...,...,...
3141,2020,1610612764,Washington Wizards,19,36,home
3142,2020,1610612765,Detroit Pistons,7,37,away
3143,2020,1610612765,Detroit Pistons,13,36,home
3144,2020,1610612766,Charlotte Hornets,15,36,away


#### Adding a new column that calculates the "win_percentage"

In [24]:
team_level_home_game_stats['win_percentage'] = round(100 * team_level_home_game_stats['win_count'] / 
                                                     team_level_home_game_stats['team_game_count'], 2)

team_level_home_game_stats


,season,team_id,team_name,win_count,team_game_count,game_location,win_percentage
0,1946,1610610025,Chicago Stags,17,30,away,56.67
1,1946,1610610025,Chicago Stags,22,31,home,70.97
2,1946,1610610026,Cleveland Rebels,13,30,away,43.33
3,1946,1610610026,Cleveland Rebels,17,30,home,56.67
4,1946,1610610028,Detroit Falcons,8,30,away,26.67
...,...,...,...,...,...,...,...
3141,2020,1610612764,Washington Wizards,19,36,home,52.78
3142,2020,1610612765,Detroit Pistons,7,37,away,18.92
3143,2020,1610612765,Detroit Pistons,13,36,home,36.11
3144,2020,1610612766,Charlotte Hornets,15,36,away,41.67


#### Compare the home game won percentage and away game won percentage for each year by team

In [25]:
px.scatter(team_level_home_game_stats, x='season', y='win_percentage',
           color='game_location', title='1946-2020: team level game won percentage by game location')


#### Calculate the median home game won and median away game home percentages

In [26]:
cols_to_drop = ['team_id', 'team_name', 'win_count', 'team_game_count']
median_win_pct_1 = team_level_home_game_stats.drop(cols_to_drop, axis=1).groupby(['season', 'game_location'])
median_win_pct = median_win_pct_1.median().reset_index()
median_win_pct


,season,game_location,win_percentage
0,1946,away,35.710
1,1946,home,62.070
2,1947,away,48.075
3,1947,home,59.935
4,1948,away,35.575
...,...,...,...
145,2018,home,60.980
146,2019,away,40.540
147,2019,home,54.805
148,2020,away,44.440


In [27]:
px.line(median_win_pct, x='season', y='win_percentage',
        color='game_location', title='1946-2020: median of the team level game won percentage by game location')

#### Is there any season os any team having away game game won percentages greater than 50%?

In [28]:
team_level_home_game_stats.query("game_location == 'away' & win_percentage > 50").sort_values("win_percentage", ascending=False)

,season,team_id,team_name,win_count,team_game_count,game_location,win_percentage
2800,2015,1610612744,Golden State Warriors,34,41,away,82.93
1610,1995,1610612741,Chicago Bulls,33,41,away,80.49
548,1972,1610612738,Boston Celtics,33,41,away,80.49
526,1971,1610612747,Los Angeles Lakers,33,41,away,80.49
1683,1996,1610612748,Miami Heat,32,41,away,78.05
...,...,...,...,...,...,...,...
1720,1997,1610612737,Atlanta Hawks,21,41,away,51.22
892,1980,1610612756,Phoenix Suns,21,41,away,51.22
922,1981,1610612745,Houston Rockets,21,41,away,51.22
1652,1995,1610612762,Utah Jazz,21,41,away,51.22


#### How does the free throw percentage (FT%) changes over time from 1949 to 2020? For each season, which team had the best FT%?

In [29]:
# Union the home and away free throw data

query = """
    SELECT
        SEASON_ID-20000 AS season,
        TEAM_ID_HOME AS team_id,
        TEAM_NAME_HOME AS team_name,
        FT_PCT_HOME AS free_throw_percentage,
        COUNT(TEAM_ID_HOME) AS team_game_count,
        "home" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_HOME

    UNION

    SELECT
        SEASON_ID-20000 AS season,
        TEAM_ID_AWAY AS team_id,
        TEAM_NAME_AWAY AS team_name,
        FT_PCT_AWAY AS free_throw_percentage,
        COUNT(TEAM_ID_AWAY) AS team_game_count,
        "away" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_AWAY
"""

team_level_free_throw_stats = pd.read_sql(query, connection)
team_level_free_throw_stats

,season,team_id,team_name,free_throw_percentage,team_game_count,game_location
0,1946,1610610025,Chicago Stags,NaN,30,away
1,1946,1610610025,Chicago Stags,NaN,31,home
2,1946,1610610026,Cleveland Rebels,NaN,30,home
3,1946,1610610026,Cleveland Rebels,0.667,30,away
4,1946,1610610028,Detroit Falcons,NaN,30,home
...,...,...,...,...,...,...
3141,2020,1610612764,Washington Wizards,0.696,36,away
3142,2020,1610612765,Detroit Pistons,0.375,37,away
3143,2020,1610612765,Detroit Pistons,0.719,36,home
3144,2020,1610612766,Charlotte Hornets,0.533,36,away


#### Calculate the free_throw_percentage rows that have NA values

In [30]:
team_level_free_throw_stats.isna()['free_throw_percentage'].sum()

212

#### Droping NA values

In [31]:
team_level_free_throw_stats = team_level_free_throw_stats.dropna().query("free_throw_percentage < 1").reset_index(drop=True)

#### Calculate median free throw percentage

In [32]:
cols_to_drop = ['team_id', 'team_name', 'team_game_count']

median_ft_pct_1 = team_level_free_throw_stats.drop(cols_to_drop, axis=1).groupby(['season', 'game_location'])
median_ft_pct = median_ft_pct_1.median().reset_index()
median_ft_pct

,season,game_location,free_throw_percentage
0,1946,away,0.6785
1,1946,home,0.5965
2,1947,away,0.7080
3,1947,home,0.6870
4,1948,away,0.6580
...,...,...,...
136,2018,home,0.7295
137,2019,away,0.7880
138,2019,home,0.7815
139,2020,away,0.7545


In [33]:
px.line(median_ft_pct, x='season', y='free_throw_percentage',
        color='game_location', title='1946-2020: median of the team level free throw percentage by game location')

#### Player background and salary analysis

#### Who are the top 10 NBA players based on 2020-2021 season salary?

In [34]:
# Getting data
# strftime -> return the date formatted according to the format string specified
# strftime('%year %month %date')

query = """
    SELECT
        namePlayer AS player_name,
        nameTeam AS team_name,
        2021 - strftime('%Y', Player_Attributes.BIRTHDATE) AS age,
        Player_Attributes.DRAFT_YEAR AS draft_year,
        2021 - Player_Attributes.DRAFT_YEAR AS years_in_NBA,
        Player_Attributes.POSITION as game_position,
        ROUND(value/1000000) AS salary_in_millions
    FROM Player_Salary
    JOIN Player ON
        Player_Salary.namePlayer = Player.full_name
    JOIN Player_Attributes ON
        Player.ID = Player_Attributes.ID
    WHERE slugSeason = '2020-21'
    ORDER BY salary_in_millions DESC
    LIMIT 10;      
"""

player_salary_top_10 = pd.read_sql(query, connection)
player_salary_top_10

,player_name,team_name,age,draft_year,years_in_NBA,game_position,salary_in_millions
0,Stephen Curry,Golden State Warriors,33,2009,12,Guard,43.0
1,James Harden,Brooklyn Nets,32,2009,12,Guard,41.0
2,John Wall,Houston Rockets,31,2010,11,Guard,41.0
3,Chris Paul,Phoenix Suns,36,2005,16,Guard,41.0
4,Russell Westbrook,Washington Wizards,33,2008,13,Guard,41.0
5,Kevin Durant,Brooklyn Nets,33,2007,14,Forward,40.0
6,LeBron James,Los Angeles Lakers,37,2003,18,Forward,39.0
7,Klay Thompson,Golden State Warriors,31,2011,10,Guard,35.0
8,Paul George,Los Angeles Clippers,31,2010,11,Forward,35.0
9,Mike Conley,Utah Jazz,34,2007,14,Guard,35.0


#### What are the game play stats for the top 10 earning NBA players based on the data available?

In [35]:
# Getting the top 10 earning player salary data

query = """
    SELECT
        namePlayer AS player_name,
        nameTeam AS team_name,
        Player_Attributes.DRAFT_YEAR AS draft_year,
        Player_Attributes.POSITION AS game_position,
        Player_Attributes.PTS AS points,
        Player_Attributes.AST AS assists,
        Player_Attributes.REB AS rebounds,
        ROUND(value/1000000) AS salary_in_millions
    FROM Player_Salary
    JOIN Player ON
        Player_Salary.namePlayer = Player.full_name
    JOIN Player_Attributes ON
        Player.ID = Player_Attributes.ID
    WHERE slugSeason = '2020-21'
    ORDER BY salary_in_millions DESC
    LIMIT 10;
"""
player_salary_top_10_game_stats = pd.read_sql(query, connection)
player_salary_top_10_game_stats

,player_name,team_name,draft_year,game_position,points,assists,rebounds,salary_in_millions
0,Stephen Curry,Golden State Warriors,2009,Guard,29.0,6.2,5.3,43.0
1,James Harden,Brooklyn Nets,2009,Guard,25.3,11.2,7.9,41.0
2,John Wall,Houston Rockets,2010,Guard,21.1,6.3,3.5,41.0
3,Chris Paul,Phoenix Suns,2005,Guard,15.7,8.8,4.7,41.0
4,Russell Westbrook,Washington Wizards,2008,Guard,21.7,10.3,9.5,41.0
5,Kevin Durant,Brooklyn Nets,2007,Forward,29.0,5.3,7.3,40.0
6,LeBron James,Los Angeles Lakers,2003,Forward,25.4,7.9,7.9,39.0
7,Klay Thompson,Golden State Warriors,2011,Guard,21.5,2.4,3.8,35.0
8,Paul George,Los Angeles Clippers,2010,Forward,22.5,5.5,6.0,35.0
9,Mike Conley,Utah Jazz,2007,Guard,16.4,5.5,3.4,35.0


## Team Background Data Analysis

#### What are the top 10 paying teams ins 2020-21 season based on the average salary?

In [36]:
query = """
    SELECT
        nameTeam AS team_name,
        ROUND(AVG(value/1000000), 2) AS avg_salary_in_millions
    FROM Player_Salary
    WHERE slugSeason = '2020-21'
    GROUP BY team_name
    ORDER BY avg_salary_in_millions DESC
    LIMIT 10;
"""

team_top_10_paying = pd.read_sql(query, connection)
team_top_10_paying


,team_name,avg_salary_in_millions
0,Golden State Warriors,12.05
1,Los Angeles Clippers,9.76
2,Philadelphia 76ers,9.11
3,Miami Heat,8.79
4,Portland Trail Blazers,8.66
5,Minnesota Timberwolves,8.64
6,Denver Nuggets,8.55
7,Utah Jazz,8.50
8,Brooklyn Nets,8.28
9,New Orleans Pelicans,8.27


In [37]:
px.bar(team_top_10_paying, x='team_name', y='avg_salary_in_millions',
       text='avg_salary_in_millions', title='top 10 high paying NBA team by average salary')